Здесь происходит обучение и оценка нейросети.

In [1]:
!pip install tensorflow
!pip install ml_metrics
!pip install nptyping

На выходе такая сеть дает вероятностное распределение по множеству рекомендованных видео. При обучении необходимо в качестве выхода задать возможные распределения для сети. В качестве таких распределений было решено взять долю пользователей, слушающих исполнителя, для которого составлен вектор, и исполнителей из множества для рекомендаций.
При таком представлении нет смысла в качестве фич подавать матрицу смежности, рассмотренную длякластеризации, так как по факту на выходе будет она же, только нормированная по строкам, поэтому рассматриваются остальные признаки.


In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix, load_npz
import ml_metrics
from nptyping import NDArray
from typing import Any, Dict
import re
import sys
import tensorflow as tf
import math

DATA_DIR = './drive/My Drive/Colab Notebooks/VK_internship/'

In [0]:
def get_network_recommendations_and_calculate_metrics(
    logits, 
    user_artists_df: pd.DataFrame(columns=["user_id", "artists"]), 
    test: NDArray[(Any,), int],
    artists_idxs_for_rec: NDArray[(1000,), int]
) -> Dict[int, float]:
    """
    Извечение рекомендаций из распределений, оценка качества рекомендаций.
    """
    metrics = {}
    for k in [1,5,10,20]:
        metrics[k]=0

    for artist_id, artist_logit in zip(test, logits):
        top21col_id = np.argpartition(artist_logit, -21)[-21:]
        recommend = artists_idxs_for_rec[list(top21col_id)]
        # Поиск списков исполнителей, которых слушают те же пользователи, 
        # что и artist_id:
        users = np.unique(user_artists_df.loc[user_artists_df["artists"]==artist_id, "user_id"].values)
        similar_artists = user_artists_df.loc[user_artists_df["user_id"].isin(users), "artists"].drop_duplicates().values
        similar_artists = similar_artists[similar_artists!=artist_id]

        # Подсчет метрик качества
        for key in metrics.keys():
            metrics[key] += ml_metrics.apk(
                actual=list(similar_artists),
                predicted=list(recommend[:key]),
                k=key
            )
    return metrics
    

В качестве признаков используется сжатая информация (SVD) о лайках, тегах и плейлистах. Информация сильно сжата из-за ограничений дискового пространства (Google диск предоставляет всего 15 Гб).
Было бы неплохо попробовать определить оптимальное знаение для SVD, но на это потребуется очень много времени.
Также было бы неплохо попробовать эи фичи для KMeans.

In [0]:
user_artists_df = pd.read_csv(DATA_DIR+"users_artists.csv", usecols=["user_id", "artists"]).drop_duplicates()
artists_id = user_artists_df["artists"].unique()
artists_id.sort()
train, test = train_test_split(artists_id, test_size=0.1)
del artists_id

partial_user_artists_df = user_artists_df.loc[user_artists_df["artists"].isin(train)]
partial_artists = partial_user_artists_df.groupby("artists").count().reset_index()
partial_artists = partial_artists.sort_values(by="user_id", ascending = False)
partial_artists_rows = partial_artists.loc[:,"artists"].values
partial_artists = partial_artists_rows[:1000]

На выходе такая сеть дает вероятностное распределение по множеству рекомендованных видео. При обучении необходимо в качестве выхода задать возможные распределения для сети. В качестве таких распределений было решено взять долю пользователей, слушающих исполнителя, для которого составлен вектор, и исполнителей из множества для рекомендаций.
При таком представлении нет смысла в качестве фич подавать матрицу смежности, рассмотренную длякластеризации, так как по факту на выходе будет она же, только нормированная по строкам, поэтому рассматриваются остальные признаки.

In [0]:
likes_matrix = np.loadtxt(DATA_DIR+"likes_matrix.csv")
playlists_matrix = np.loadtxt(DATA_DIR+"playlists_matrix.csv")
full_matrix = np.hstack((likes_matrix, playlists_matrix))
del likes_matrix, playlists_matrix
user_tags_matrix = np.loadtxt(DATA_DIR+"user_tags_matrix.csv")
full_matrix = np.hstack((full_matrix, user_tags_matrix))
del user_tags_matrix

In [0]:
user_artists_df = pd.read_csv(DATA_DIR+"users_artists.csv", usecols=["user_id", "artists"]).drop_duplicates()
artists_id = user_artists_df["artists"].unique()
artists_id.sort()
train, test = train_test_split(artists_id, test_size=0.1)
del artists_id
partial_user_artists_df = user_artists_df.loc[user_artists_df["artists"].isin(train)]
partial_artists = partial_user_artists_df.groupby("artists").count().reset_index()
partial_artists = partial_artists.sort_values(by="user_id", ascending = False)
partial_artists = partial_artists.loc[:,"artists"].values
partial_artists = partial_artists[:1000]
# строки в матрице вероятности отсортированы по популярности пользователей, это надо исправить

In [0]:
train_probabilities_matrix = load_npz(DATA_DIR+"train_matrix.npz") # y 
train_probabilities_matrix = Normalizer("l1").fit_transform(train_probabilities_matrix) 
# нормализация строк от 0 до 1, чтобы было похоже на распределение вероятностей

In [0]:
train_matrix = full_matrix[partial_artists_rows,:]
train_matrix = train_matrix[:train_probabilities_matrix.shape[0],:]
test_matrix = full_matrix[test, :]
del full_matrix

Х - векторные представления, 

у - распределения вероятностей, основанное на взаимодействиях пользователей 

In [0]:
vector_length = train_matrix.shape[1]
recommended_artists_amount = train_probabilities_matrix.shape[1]

In [0]:
the_least_elem = np.min([np.min(train_matrix), np.min(test_matrix)])

In [0]:
train_matrix = train_matrix-the_least_elem #make positive

Чтобы попробовать исправить внутренние ошибки, возникающие при предсказании модели, было принято решение упростить модель, поэтому слои имеют размерность 64 в отличии от статьи, где размерность составляет 256.

In [0]:
with tf.device('/GPU:0'):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Embedding(input_dim=int(math.ceil(np.max([np.max(train_matrix), np.max(test_matrix)])))+1, 
                                        input_length=vector_length,
                                        output_dim=64))
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.GRU(64, dropout=0.1))
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(recommended_artists_amount, activation='softmax'))
    model.compile(optimizer="AdaGrad", loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True))

In [0]:
train_probabilities_matrix = csr_matrix(train_probabilities_matrix)

Обучение по батчам:

In [0]:
batch_size = 1000
batches_amunt = int(math.ceil(train_matrix.shape[0]/batch_size))
with tf.device('/GPU:0'):
    for batch_counter in range(batches_amunt):
        train_y = train_probabilities_matrix[batch_counter*batch_size:(batch_counter+1)*batch_size].toarray()
        train_x = train_matrix[batch_counter*batch_size:(batch_counter+1)*batch_size, :]

        model.train_on_batch(x=train_x, y=train_y)
        model.save(DATA_DIR + 'recommendational_network.h5')
        print(batch_counter)


In [0]:
model = tf.keras.models.load_model(DATA_DIR + 'recommendational_network.h5')
test_matrix = test_matrix-the_least_elem

Метрики:

В ходе выполнения возникли ошибки, на исправление времени не хватило, так как опыт работы с tensorflow у меня невелик.

In [18]:
metrics = {}
for k in [1,5,10,20]:
    metrics[k]=0

batch_size = 1000
batches_amount = int(math.ceil(train_matrix.shape[0]/batch_size))+1


for batch_counter in range(batches_amount):
    logits = model(test_matrix[batch_counter*batch_size:(batch_counter+1)*batch_size, :])
    batch_metrics = get_network_recommendations_and_calculate_metrics(
            logits, 
            user_artists_df, 
            test[batch_counter*batch_size:(batch_counter+1)*batch_size],
            partial_artists
    )
    for key in metrics.keys():
          metrics[key]+=batch_metrics[key]

UnknownError: ignored

In [0]:
for key in metrics.keys():
    metrics[key]/=len(test)
    print("MAP@{}: ".format(key), metrics[key])